<a href="https://colab.research.google.com/github/fromakim/2021Election_Analysis/blob/main/data_prepreocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>